In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.dates as mdates
import scipy
import os
from scipy.stats import linregress
from datetime import timedelta

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker

plt.rcParams['figure.dpi'] = 300

## 1. Load in and organize LAI data

In [2]:
lai_data_dir = 'C:/Users/jeanallen/Desktop/SHIFT_CWC/data/field_data'

lai_1 = pd.read_csv(os.path.join(lai_data_dir, 'LAI_20220427_20220428.csv'))
lai_2 = pd.read_csv(os.path.join(lai_data_dir, 'LAI_20220518_20220521.csv'))
lai_3 = pd.read_csv(os.path.join(lai_data_dir, 'LAI_20220526_20220527.csv'))
lai_4 = pd.read_csv(os.path.join(lai_data_dir, 'LAI_20220912_20220921_backup.csv'))

In [3]:
# make list of all trees measured at some point over the course of the study
all_trees = lai_1['Tree Num'].unique().tolist() + lai_2['Tree Num'].unique().tolist() + lai_3['Tree Num'].unique().tolist() + lai_4['Tree Num'].unique().tolist()
all_trees = list(set(all_trees))
print('Number of LAI trees measured:', len(all_trees))

Number of LAI trees measured: 108


In [4]:
# functions to go grab LAI values for a given tree number
def getlai1(tree_num):
    try:
        return lai_1[lai_1['Tree Num'] == tree_num]['LAI'].values[0]
    except IndexError:
        return None
def getlai2(tree_num):
    try:
        return lai_2[lai_2['Tree Num'] == tree_num]['LAI'].values[0]
    except IndexError:
        return None
def getlai3(tree_num):
    try:
        return lai_3[lai_3['Tree Num'] == tree_num]['LAI'].values[0]
    except IndexError:
        return None
def getlai4(tree_num):
    try:
        return lai_4[lai_4['Tree Num'] == tree_num]['LAI'].values[0]
    except IndexError:
        return None

In [5]:
lai_4

,Record Type,Date and Time,Tree Num,Average Above PAR,Average Below PAR,Leaf Distribution [Χ],Zenith Angle,Latitude,Longitude,Zenith,Fb,K,Tau,LAI
0,SUM,9/12/2022 11:21,2024,1922.924805,510.1,1,37,34,-120,36.716522,0.905367,0.628658,0.265273,2.188278
1,SUM,9/12/2022 11:23,2023,1954.650879,371.8,1,36,34,-120,36.457508,0.905367,0.626551,0.190213,2.744907
2,SUM,9/12/2022 11:31,2022,1827.136230,397.6,1,35,34,-120,35.434505,0.905367,0.618492,0.217608,2.551428
3,SUM,9/12/2022 11:36,2021,1733.679199,185.1,1,35,34,-120,34.846302,0.905367,0.614039,0.106767,3.766719
4,SUM,9/12/2022 11:38,2025,1719.531250,418.1,1,35,34,-120,34.596474,0.904603,0.612186,0.243148,2.386259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,SUM,9/21/2022 14:13,2316,1630.407715,605.4,1,38,34,-120,37.882778,0.896431,0.638482,0.371318,1.603648
103,SUM,9/21/2022 14:14,2315,1627.001953,545.7,1,38,34,-120,38.002861,0.896109,0.639526,0.335402,1.765494
104,SUM,9/21/2022 14:15,2311/2312,1623.266602,362.0,1,38,34,-120,38.139812,0.895810,0.640725,0.223007,2.420737
105,SUM,9/21/2022 14:16,2313,1620.434570,971.6,1,38,34,-120,38.254387,0.895689,0.641734,0.599592,0.823994


In [6]:
# create a dataframe with all the LAI values for each tree
lai_df = pd.DataFrame(all_trees, columns=['Tree Num'])
lai_df['LAI_1'] = lai_df['Tree Num'].apply(getlai1)
lai_df['LAI_2'] = lai_df['Tree Num'].apply(getlai2)
lai_df['LAI_3'] = lai_df['Tree Num'].apply(getlai3)
lai_df['LAI_4'] = lai_df['Tree Num'].apply(getlai4)

# drop na
lai_df = lai_df.dropna()

# melt it up...
lai_df = lai_df.melt(id_vars='Tree Num', var_name='Date', value_name='LAI')

# rename Tree Num to just Tree
lai_df = lai_df.rename(columns={'Tree Num': 'Tree'})
lai_df

,Tree,Date,LAI
0,2376,LAI_1,3.920365
1,2323,LAI_1,2.585485
2,2371,LAI_1,2.409970
3,2367,LAI_1,1.648310
4,2354,LAI_1,2.684098
...,...,...,...
251,2023,LAI_4,2.744907
252,2356,LAI_4,2.392786
253,2310,LAI_4,1.190060
254,2302,LAI_4,2.307099


## 2. ANOVA

In [7]:
# create a column with just the number out of the LAI column so AnovaRM doesn't get mad at me
lai_df['Num'] = lai_df['Date'].apply(lambda x: int(x.split('_')[1]))

from statsmodels.stats.anova import AnovaRM 

# repeated measures ANOVA
anova = AnovaRM(data=lai_df, depvar='LAI', within=['Num'], subject='Tree')

res = anova.fit()
print(res)

              Anova
    F Value Num DF  Den DF  Pr > F
----------------------------------
Num  5.6514 3.0000 189.0000 0.0010



In [8]:
from pingouin import pairwise_tests

posthoc_results = pairwise_tests(data=lai_df, dv='LAI', within='Num', subject='Tree', padjust='bonferroni')
print(posthoc_results)

  Contrast  A  B  Paired  Parametric         T   dof alternative     p-unc  \
0      Num  1  2    True        True -1.209394  63.0   two-sided  0.231031   
1      Num  1  3    True        True -0.595713  63.0   two-sided  0.553501   
2      Num  1  4    True        True  2.477823  63.0   two-sided  0.015912   
3      Num  2  3    True        True  0.605816  63.0   two-sided  0.546813   
4      Num  2  4    True        True  3.212317  63.0   two-sided  0.002075   
5      Num  3  4    True        True  2.481121  63.0   two-sided  0.015778   

     p-corr    p-adjust    BF10    hedges  
0  1.000000  bonferroni   0.274 -0.092882  
1  1.000000  bonferroni   0.162 -0.042653  
2  0.095470  bonferroni   2.293  0.328751  
3  1.000000  bonferroni   0.163  0.046462  
4  0.012453  bonferroni  13.623  0.413933  
5  0.094671  bonferroni   2.309  0.352577  
